# Spacecraft Attitude Control - PD

## Python Control System Toolbox
If you are running this code at your local computer where the python control system toolbox is already installed skip or uncomment the following line.

In [ ]:
#pip install control

## Useful Tools

We define teh following three functions that are useful for control system design:

*   `zeta = mae4182.Mp2zeta(Mp)`: convert the overshoot into the correspondin value of damping for the prototype second order system;
* `mae4182.sgrid(zeta, wn)`: draw the line of the fixed damping and the circle of the fixed natural frequency in the complex plane;
* `M_p, t_r, t_s, t_d = mae4182.step_info(sysG)`: compute the time domain specifications according to the definition of the class.

In [ ]:
import control
import matplotlib.pyplot as plt
import numpy as np
import sys
import types

mae4182 = types.ModuleType('mae4182')
sys.modules['mae4182'] = mae4182

mae4182_code = '''
import control
import matplotlib.pyplot as plt
import numpy as np

import sys
import types

def Mp2zeta(Mp):
    c=np.log(Mp)**2 
    zeta = np.sqrt(c/(np.pi**2+c))                 
    return zeta


def sgrid(zeta,wn):
    axes = plt.gca()
    xmin, xmax = axes.get_xlim()
    ymin, ymax = axes.get_ylim()
    
    theta=np.linspace(0,2*np.pi,501)
    axes.plot(wn*np.cos(theta),wn*np.sin(theta),'k:')
    
    if zeta < 1:
        tan_theta=zeta/np.sqrt(1-zeta**2)
        
    if xmin < -tan_theta*ymax:
        axes.plot([0, -tan_theta*ymax],[0, ymax],'k:')
        axes.plot([0, -tan_theta*ymax],[0, ymin],'k:')
    else:
        axes.plot([0, xmin],[0, -xmin/tan_theta],'k:')
        axes.plot([0, xmin],[0, xmin/tan_theta],'k:')


def step_info(sysG):
    output = control.step_info(sysG, SettlingTimeThreshold = 0.05)
    M_p = output["Overshoot"]
    t_r = output["RiseTime"]
    t_s = output["SettlingTime"]
    output = control.step_info(sysG, RiseTimeLimits = (0,0.5))
    t_d = output["RiseTime"]
    return M_p, t_r, t_s, t_d

'''
exec(mae4182_code, mae4182.__dict__)




## Matplotlib Backend

Matplotlib has several backends when generating plots:

* `%matplotlib inline`: normal mode (used for step response)
* `%matplotlib widget`: interactive mode (used for root locus and animation)

The mode should be changed properly for the task of the cell. After changing the mode, a new figure should be generated by `plt.figure()` to activate the selected mode properly.


## Dynamics of Satellite

Define the system tranfer function $G(s) = \dfrac{d}{Js^2+cs}$, where $J=1$, $d=1$, and $c=0$.

In [ ]:
J=1
d=1
c=0

sysG = control.tf(d,[J, c, 0])
display(sysG)

The poles are at $p_{1,2}=0$. Thus, this system is not stable.

In [ ]:
%matplotlib inline

N = 501
t = np.linspace(0,10,N)
t, y = control.step_response(sysG, t)
plt.figure()
plt.plot(t,y)
plt.xlabel('t')
plt.ylabel('y')
plt.title('step response')
plt.grid()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import matplotlib.animation as animation
import matplotlib.patches as patches

%matplotlib widget

theta = y

# generate animation
w = 1
h = 2
b = 1.2
fw = 0.3
fb = 1.5
vertices_left = np.array([ [-w,h], [-w, -b], [-fw*w, -b], [-w, -fb*b] ]).T
vertices_right = np.array([[-1, 0], [0, 1]])@vertices_left
vertices0 = np.concatenate((vertices_left, np.fliplr(vertices_right)), axis=1)


R = np.array([[np.cos(theta[0]), np.sin(theta[0])], [-np.sin(theta[0]), np.cos(theta[0])]])
vertices = R @ vertices0

patch = Polygon(vertices.T, facecolor = 'b')

fig = plt.figure(figsize = (5,5))
ax = plt.gca()
ax.add_patch(patch)
ax.set_xlim([-3,3])
ax.set_ylim([-3,3])
ax.axis('off')
plt.show()

def init():
    return patch,

def animate(i):
    R = np.array([[np.cos(theta[i]), np.sin(theta[i])], [-np.sin(theta[i]), np.cos(theta[i])]])
    vertices = R @ vertices0

    patch.set_xy(vertices.T)
    return patch,

ani = animation.FuncAnimation(fig, animate, N, init_func=init, interval=10, repeat=False)




## Time-Domain Control Design



We wish to desgin a control system such that
* $M_p \leq 0.2$
* $t_r \leq 1.5$.


First, convert design specifications into desired pole location:

In [ ]:
Mp = 0.2
tr = 1.5

zeta = mae4182.Mp2zeta(Mp)
wn = (0.8+2.5*zeta)/tr

print(f'zeta = {zeta:.2f}, wn={wn:.2f}')

## P Control
Let the controller transfer function be $C(s)=K$. The characteristic equation for the closed-loop system is $1+C(s)G(s)= 1+ K G(s)=0$. Therefore, the rool locus represetning the closed-loop pole location for varying $K$ is generated as follows. 

The root locus is generated as follows.

In [ ]:
# the following line is to draw the root locus in the interactive mode
# we need the interactive mode to select the desired pole location
%matplotlib widget

plt.figure()
roots, K = control.root_locus(sysG, xlim=[-8,1], ylim=[-3,3], grid=False)
mae4182.sgrid(zeta,wn)




The pole of the controlled system is on the imaginary axis for any $K$.

## PD Control

The controller transfer function is $C(s)= K+ K_d s = K(1+ T_ds)$. The characteristic equation for the closed-loop system is $1+C(s)G(s)= 1+ K(1+T_s s) G(s)= 1+ KC_0(s)G(s)=0$, where $C_0(s) = T_d s+1$. 

We first choose $T_d$, then generate root locus with respect to $K$ as follows.

In [ ]:
# %matplotlib inline
# plt.figure()

%matplotlib widget

Td = 1.0
sysC0 = control.tf([Td, 1],1)
plt.figure()
roots, K = control.root_locus(sysC0*sysG,xlim=[-4,1],ylim=[-3,3],grid=False)
axes = plt.gca()
axes.set_title('T_d='+str(Td))
   
mae4182.sgrid(zeta, wn)


We simulate the step response and find the time-domain specs. 

In [ ]:
# switch back to normal plotting mode
%matplotlib inline

K = 2.47
Td = 1.0

Kd = K*Td
sysC = control.tf([Kd, K],1)
sysYR = control.feedback(sysC*sysG,1)
t, y = control.step_response(sysYR)
plt.figure()
plt.plot(t,y)
plt.xlabel('t')
plt.ylabel('y')
plt.title('step response')
plt.grid()


M_p, t_r, t_s, t_d = mae4182.step_info(sysYR)
print(f'M_p = {M_p:.2f}')
print(f't_r = {t_r:.2f}')


What if we wish to rotate the satellite by $90^\circ$. From the above, we have $y_{ss} = 1$ when $r=u_s(t)$. 

According to the principle of superpostiion of the linear syteme, if we choose $r=\frac{\pi}{2}u_s(t)$, then we will get $y_{ss} = \frac{\pi}{2}$. In other words, the reference input can be scaled accordingly. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import matplotlib.animation as animation
import matplotlib.patches as patches

%matplotlib widget

N = 500
t = np.linspace(0, 10, N)
t, y = control.forced_response(sysYR, t, np.pi/2)

# generate animation
w = 1
h = 2
b = 1.2
fw = 0.3
fb = 1.5
vertices_left = np.array([ [-w,h], [-w, -b], [-fw*w, -b], [-w, -fb*b] ]).T
vertices_right = np.array([[-1, 0], [0, 1]])@vertices_left
vertices0 = np.concatenate((vertices_left, np.fliplr(vertices_right)), axis=1)

#theta = np.linspace(0, 50, 100)*np.pi/180
theta = y

R = np.array([[np.cos(theta[0]), np.sin(theta[0])], [-np.sin(theta[0]), np.cos(theta[0])]])
vertices = R @ vertices0

patch = Polygon(vertices.T, facecolor = 'b')

fig = plt.figure(figsize = (5,5))
ax = plt.gca()
ax.add_patch(patch)
ax.set_xlim([-3,3])
ax.set_ylim([-3,3])
ax.axis('off')
plt.show()

def init():
    return patch,

def animate(i):
    R = np.array([[np.cos(theta[i]), np.sin(theta[i])], [-np.sin(theta[i]), np.cos(theta[i])]])
    vertices = R @ vertices0

    patch.set_xy(vertices.T)
    return patch,

ani = animation.FuncAnimation(fig, animate, N, init_func=init, interval=20, repeat=False)


